In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

####Data Reading

In [0]:
df=spark.sql("select distinct RatecodeID from databricks_catalog.silver.nyc_taxi_trips where RatecodeID is not null order by RatecodeID")
df.display()

####Lookup table 

In [0]:
%sql
create table if not exists databricks_catalog.gold.rate_code_desc 
(RateCodeId int, RateCodeDesc string);

insert into databricks_catalog.gold.rate_code_desc (RateCodeId, RateCodeDesc) VALUES (1,'Standard rate'),
(2,'JFK'),
(3,'Newark'),
(4,'Nassau or Westchester'),
(5,'Negotiated fare'),
(6,'Group ride'),
(99,'Null/unknown')

In [0]:
df_rate_desc=spark.sql("select * from databricks_catalog.gold.rate_code_desc")
df_rate_desc.display()

In [0]:
df=df.join(df_rate_desc,on=['RatecodeID'],how='left')\
    .withColumn('rate_sk',col("RatecodeID"))\
        .dropDuplicates(['rate_sk'])

In [0]:
df.display()


####Upsert logic

In [0]:
from delta.tables import DeltaTable
if spark.catalog.tableExists("databricks_catalog.gold.dim_rate_code_scd1"):
  dt=DeltaTable.forName(spark,"databricks_catalog.gold.dim_rate_code_scd1")
  (dt.alias("t").merge(
    df.alias("s"),
    "t.rate_sk=s.rate_sk"
  ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute())

else:
  df.write.format('delta').mode('overwrite').saveAsTable("databricks_catalog.gold.dim_rate_code_scd1")